In [11]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"annadaznabila","key":"f8abfd40d4ae4072d597a1bed5c3f63a"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [3]:
!kaggle datasets download -d vivekyash/heart-failure-prediction-deep-learning-ann

Dataset URL: https://www.kaggle.com/datasets/vivekyash/heart-failure-prediction-deep-learning-ann
License(s): unknown
  0% 0.00/738k [00:00<?, ?B/s]
100% 738k/738k [00:00<00:00, 76.3MB/s]


In [7]:
!unzip heart-failure-prediction-deep-learning-ann.zip -d heart-failure-prediction-deep-learning-ann
!ls heart-failure-prediction-deep-learning-ann

Archive:  heart-failure-prediction-deep-learning-ann.zip
  inflating: heart-failure-prediction-deep-learning-ann/Heart Failure Prediction_ Deep Learning - ANN(Vivek).pdf  
  inflating: heart-failure-prediction-deep-learning-ann/heart_failure_clinical_records_dataset.csv  
 heart_failure_clinical_records_dataset.csv
'Heart Failure Prediction_ Deep Learning - ANN(Vivek).pdf'


In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [15]:
df = pd.read_csv("/content/heart-failure-prediction-deep-learning-ann/heart_failure_clinical_records_dataset.csv")

In [16]:
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    int64  
 10  smoking                   299 non-null    int64  
 11  time                      299 non-null    int64  
 12  DEATH_EVENT               299 non-null    int64  
dtypes: float64(3), int64(10)
memory usage: 30.5 KB


In [18]:
df = df.drop(columns=['creatinine_phosphokinase','ejection_fraction','platelets','serum_creatinine'])

In [19]:
df.head()

,age,anaemia,diabetes,high_blood_pressure,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,0,1,130,1,0,4,1
1,55.0,0,0,0,136,1,0,6,1
2,65.0,0,0,0,129,1,1,7,1
3,50.0,1,0,0,137,1,0,7,1
4,65.0,1,1,0,116,0,0,8,1


In [58]:
X = df.drop("DEATH_EVENT",axis=1)
y = pd.get_dummies(df['DEATH_EVENT'])

In [60]:
from sklearn import preprocessing

In [61]:
col_names = list(X.columns)
s_scaler = preprocessing.StandardScaler()
X_df= s_scaler.fit_transform(X)
X_df = pd.DataFrame(X_df, columns=col_names)
X_df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,299.0,5.703353e-16,1.001676,-1.754448,-0.828124,-0.070223,0.771889,2.877170
anaemia,299.0,1.009969e-16,1.001676,-0.871105,-0.871105,-0.871105,1.147968,1.147968
diabetes,299.0,9.060014e-17,1.001676,-0.847579,-0.847579,-0.847579,1.179830,1.179830
high_blood_pressure,299.0,0.000000e+00,1.001676,-0.735688,-0.735688,-0.735688,1.359272,1.359272
serum_sodium,299.0,-8.673849e-16,1.001676,-5.363206,-0.595996,0.085034,0.766064,2.582144
sex,299.0,-8.911489e-18,1.001676,-1.359272,-1.359272,0.735688,0.735688,0.735688
smoking,299.0,-1.188199e-17,1.001676,-0.687682,-0.687682,-0.687682,1.454161,1.454161
time,299.0,-1.901118e-16,1.001676,-1.629502,-0.739000,-0.196954,0.938759,1.997038


In [62]:
X_train, X_test, y_train,y_test = train_test_split(X_df,y,test_size=0.25,random_state=7)

In [63]:
input_dim = X.shape[1]

In [65]:
from keras.layers import Dense, BatchNormalization, Dropout, LSTM
from keras.models import Sequential

In [66]:
model = Sequential()
model.add(Dense(8, input_dim = input_dim, kernel_initializer='normal', activation='relu'))
model.add(Dense(5, activation = "relu", kernel_initializer='normal'))
model.add(Dense(2, activation = 'sigmoid'))

In [67]:
from tensorflow.keras.optimizers import Adam

In [68]:
model.compile(optimizer = Adam(learning_rate = 0.001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [69]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 8)                 72        
                                                                 
 dense_25 (Dense)            (None, 5)                 45        
                                                                 
 dense_26 (Dense)            (None, 2)                 12        
                                                                 
Total params: 129 (516.00 Byte)
Trainable params: 129 (516.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [70]:
result = model.fit(X_train, y_train, epochs = 15, batch_size = 10)

Epoch 1/15
23/23 [==============================] - 1s 3ms/step - loss: 0.6901 - accuracy: 0.6429
Epoch 2/15
23/23 [==============================] - 0s 2ms/step - loss: 0.6827 - accuracy: 0.6518
Epoch 3/15
23/23 [==============================] - 0s 3ms/step - loss: 0.6733 - accuracy: 0.6518
Epoch 4/15
23/23 [==============================] - 0s 3ms/step - loss: 0.6615 - accuracy: 0.6518
Epoch 5/15
23/23 [==============================] - 0s 4ms/step - loss: 0.6439 - accuracy: 0.7188
Epoch 6/15
23/23 [==============================] - 0s 4ms/step - loss: 0.6223 - accuracy: 0.7634
Epoch 7/15
23/23 [==============================] - 0s 3ms/step - loss: 0.5982 - accuracy: 0.7723
Epoch 8/15
23/23 [==============================] - 0s 4ms/step - loss: 0.5727 - accuracy: 0.7812
Epoch 9/15
23/23 [==============================] - 0s 4ms/step - loss: 0.5486 - accuracy: 0.8036
Epoch 10/15
23/23 [==============================] - 0s 5ms/step - loss: 0.5254 - accuracy: 0.8214
Epoch 11/15
23/23 [

In [71]:
y_pred =model.predict(X_test)
y_pred = (y_pred>0.5)

3/3 [==============================] - 0s 5ms/step


In [73]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score

In [74]:
accuracy = accuracy_score(y_test,y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.7733333333333333


In [75]:
model.save('./heartfailure.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [79]:
model = tf.keras.models.load_model('./heartfailure.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open("./heartfailure.tflite", "wb").write(tflite_model)

2652

In [80]:
df.head(50)

,age,anaemia,diabetes,high_blood_pressure,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,0,1,130,1,0,4,1
1,55.0,0,0,0,136,1,0,6,1
2,65.0,0,0,0,129,1,1,7,1
3,50.0,1,0,0,137,1,0,7,1
4,65.0,1,1,0,116,0,0,8,1
5,90.0,1,0,1,132,1,1,8,1
6,75.0,1,0,0,137,1,0,10,1
7,60.0,1,1,0,131,1,1,10,1
8,65.0,0,0,0,138,0,0,10,1
9,80.0,1,0,1,133,1,1,10,1


In [81]:
model.predict([[55.0,0,0,0,136,1,0,6]])

1/1 [==============================] - 0s 132ms/step


array([[1.0000000e+00, 1.5028274e-05]], dtype=float32)